In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
# Гиперпараметры
input_size = 3 * 32 * 32  # Изображения CIFAR-10 имеют размеры 3x32x32
num_classes = 10
learning_rate = 0.001
batch_size = 100
num_epochs = 5

In [4]:
# Преобразования для данных
transform = transforms.Compose([
    transforms.ToTensor(),  # Преобразование изображений в тензор
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация изображений
])

In [5]:
# Загрузка датасета CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# Определение модели перцептрона
class Perceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)  # Полносвязный слой

    def forward(self, x):
        out = self.fc(x)
        return out

In [7]:
hidden_size = 128
# Определение модели
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 512)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(512, 512)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(512, 256)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(256, hidden_size)
        self.relu6 = nn.ReLU()
        self.fc7 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        out = self.relu5(out)
        out = self.fc6(out)
        out = self.relu6(out)
        out = self.fc7(out)
        return out

In [8]:
model = MLP(input_size, hidden_size, num_classes)  # Перемещаем модель на GPU, если доступно

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Обучение модели
for epoch in range(num_epochs):
    for images, labels in tqdm(train_loader):
        images = images.view(-1, input_size)  # Преобразование изображений в вектор и перемещение на GPU
        labels = labels

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 45.78it/s]


Epoch [1/5], Loss: 2.0719


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:13<00:00, 37.57it/s]


Epoch [2/5], Loss: 1.7997


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 35.16it/s]


Epoch [3/5], Loss: 1.7461


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 35.15it/s]


Epoch [4/5], Loss: 1.8514


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 34.98it/s]

Epoch [5/5], Loss: 1.7156


In [10]:
# Тестирование модели
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, input_size)
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

# Обучение завершено

Accuracy of the model on the 10000 test images: 42.36 %
